<a href="https://colab.research.google.com/github/smuggypusss/NLP/blob/main/FakeNewsLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
Dataset: 'https://www.kaggle.com/c/fake-news/data#'

In [ ]:
df=pd.read_csv('train.csv',error_bad_lines=False,engine='python')

<ipython-input-9-c76bdf37a4d4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',error_bad_lines=False,engine='python')
Skipping line 4650: unexpected end of data


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id          0
title     128
author    452
text       11
label       0
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.head(

)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
x=df.drop('label',axis=1)

In [ ]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
4643,4643,The Hyperinflationary Death Watch,The Doc,Home » Headlines » Finance News » The Hyperinf...
4644,4644,Taxpayer Funded Peer Lord Kerr Calls Taxpayers...,dailouk,Home | World | Taxpayer Funded Peer Lord Kerr ...
4645,4645,Kidnapped Border Patrol Agent Was Hacked with ...,Brandon Darby &amp Ildefonso Ortiz,Suspects in the kidnapping and attack of a U. ...
4646,4646,NRA to Campaign Against Red State Democrats wh...,AWR Hawkins,With a vote on the confirmation of Supreme Cou...


In [ ]:
y=df['label']

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=5000


In [ ]:
message=x.copy()

In [ ]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
#message.reset_index(inplace=True)

In [ ]:
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
###Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
  print(i)
  review=re.sub('[^a-zA-Z]',' ',message['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3473, 17, 3818, 3203, 1402, 1161, 3407, 661, 650, 3325],
 [3773, 3994, 1415, 3474, 2103, 482, 1036],
 [4878, 2552, 1551, 1646],
 [1530, 332, 1556, 3251, 189, 1702],
 [2999, 2103, 200, 324, 1849, 2704, 2103, 1651, 1907, 1400],
 [3885,
  268,
  1864,
  680,
  1517,
  978,
  4845,
  4582,
  2415,
  2249,
  1652,
  4145,
  4508,
  2361,
  1036],
 [350, 2149, 4198, 1673, 3695, 2128, 1611, 564, 3053, 3447, 4564],
 [2178, 3321, 3626, 2913, 3838, 4367, 978, 4466, 3053, 3447, 4564],
 [3240, 3669, 559, 1140, 3470, 2733, 64, 100, 978, 3007],
 [416, 1602, 3248, 4488, 2662, 4817, 3615, 2525],
 [4357, 1178, 1849, 1880, 306, 4838, 2864, 2780, 2653, 3416, 1296],
 [3251, 4090, 1402, 2733, 978, 3838],
 [19, 3985, 2547, 2956, 98, 2048, 4884, 603, 4030],
 [3187, 4029, 914, 2800, 3169, 4390, 1607, 3053, 3447, 4564],
 [2059, 796, 2022, 2924, 2525, 3053, 3447, 4564],
 [2218, 11, 1491, 158, 314, 1655, 4847, 3920, 198, 2650],
 [3433, 1733, 3994],
 [294, 1838, 18, 1081, 978, 781, 4723, 1036],
 [3750, 3118, 14

Embedding

In [ ]:
sent_len=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ...  661  650 3325]
 [   0    0    0 ... 2103  482 1036]
 [   0    0    0 ... 2552 1551 1646]
 ...
 [   0    0    0 ... 2792 1678  899]
 [   0    0    0 ...  346 1727 1036]
 [   0    0    0 ... 2573    4  933]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3473,
         17, 3818, 3203, 1402, 1161, 3407,  661,  650, 3325], dtype=int32)

In [ ]:
#Creating model
embedding_vector_features=40 ##number of features representations
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(4068, (4068,))

In [ ]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
x_final.shape,y_final.shape

((4068, 20), (4068,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.4,random_state=42)


In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
39/39 [==============================] - 5s 52ms/step - loss: 0.6097 - accuracy: 0.7107 - val_loss: 0.5820 - val_accuracy: 0.6978
Epoch 2/10
39/39 [==============================] - 1s 36ms/step - loss: 0.3500 - accuracy: 0.8451 - val_loss: 0.2920 - val_accuracy: 0.8661
Epoch 3/10
39/39 [==============================] - 2s 63ms/step - loss: 0.1567 - accuracy: 0.9467 - val_loss: 0.2597 - val_accuracy: 0.8845
Epoch 4/10
39/39 [==============================] - 2s 43ms/step - loss: 0.0856 - accuracy: 0.9762 - val_loss: 0.2793 - val_accuracy: 0.8907
Epoch 5/10
39/39 [==============================] - 1s 37ms/step - loss: 0.0353 - accuracy: 0.9910 - val_loss: 0.3360 - val_accuracy: 0.8876
Epoch 6/10
39/39 [==============================] - 1s 37ms/step - loss: 0.0223 - accuracy: 0.9967 - val_loss: 0.4134 - val_accuracy: 0.8802
Epoch 7/10
39/39 [==============================] - 1s 37ms/step - loss: 0.0237 - accuracy: 0.9955 - val_loss: 0.2700 - val_accuracy: 0.9005
Epoch 8/10
39

In [ ]:
y_pred=model.predict(x_test)

51/51 [==============================] - 1s 6ms/step


In [ ]:
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[832, 104],
       [ 78, 614]])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_pred,y_test)

0.8882063882063882